In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder


In [2]:
df = pd.read_excel('train.xlsx')

In [3]:
df['Number_of_Weeks_Used'].fillna(df.Number_of_Weeks_Used.median(), inplace=True)

In [4]:
df2 = pd.get_dummies(df, columns=['Crop', 'Soil'])

In [5]:
ext = df2[df2.Crop_status == 2]
df3 = pd.concat([df2,ext,ext,ext,ext])
df3.Crop_status.value_counts()

0    66716
1    11183
2    10505
Name: Crop_status, dtype: int64

In [6]:
#dounsampling step
from sklearn.utils import resample

# Separating majority and minority classes
df_major = df3[df3.Crop_status==1]
df_minor= df3[df3.Crop_status==2]
 
# Downsample major class
df_majority_downsampled = resample(df_major,replace=False,n_samples=len(df_minor),random_state=42)
 
# Combining minor class with downsampled major class
dff = pd.concat([df_majority_downsampled, df_minor])
 
# Display new class counts
dff.Crop_status.value_counts(normalize = True)
dff.reset_index(inplace = True)
dff.drop(['index'], axis = 1, inplace = True)

#dounsampling step
from sklearn.utils import resample

# Separating majority and minority classes
df_major = df3[df3.Crop_status==0]
df_minor= dff[dff.Crop_status==2]
 
# Downsample major class
df_majority_downsampled = resample(df_major,replace=False,n_samples=len(df_minor),random_state=42)
 
# Combining minor class with downsampled major class
df4 = pd.concat([df_majority_downsampled,dff])
 
# Display new class counts
df4.Crop_status.value_counts(normalize = True)
df4.reset_index(inplace = True)
df4.drop(['index'], axis = 1, inplace = True)

In [7]:
df4

,ID,Insects,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Season,Crop_status,Crop_Feed,Crop_Food,Soil_clay,Soil_silt
0,58234,851,2,10,24.0,12,2,0,1,0,1,0
1,39048,851,2,20,33.0,4,1,0,0,1,1,0
2,28915,1296,2,80,45.0,3,2,0,0,1,1,0
3,43924,1297,3,80,43.0,0,2,0,1,0,0,1
4,41224,448,3,20,26.0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
31510,79786,1898,2,40,40.0,10,3,2,0,1,1,0
31511,79787,1898,2,40,37.0,12,2,2,0,1,1,0
31512,79865,2267,2,10,26.0,29,1,2,0,1,1,0
31513,79866,2267,2,40,38.0,10,1,2,0,1,1,0


In [8]:
df4.Crop_status.value_counts(normalize = True)

2    0.333333
1    0.333333
0    0.333333
Name: Crop_status, dtype: float64

In [12]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

In [13]:
X= df4.drop(['ID','Crop_status'],axis = 1)

In [14]:
y = df4['Crop_status']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 

In [16]:
from sklearn.ensemble import RandomForestClassifier 
dtree_model = RandomForestClassifier(max_depth = 2, random_state = 42).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
cm = confusion_matrix(y_test, dtree_predictions)
cm

array([[2031,    0,  536],
       [1346,    0, 1304],
       [ 782,    0, 1880]], dtype=int64)

In [17]:
accuracy = dtree_model.score(X_test, y_test)
accuracy

0.49638278969412364

In [28]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42).fit(X_train,y_train)
dt.score(X_test,y_test)*100

69.57735753268182

In [34]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier(random_state = 42).fit(X_train, y_train)
forest.score(X_test,y_test)*100

73.930701865719

In [36]:
pred_f = forest.predict(X_test)

In [41]:
pred_fo = pd.Series(pred_f)
pred_fo.value_counts()

2    3153
0    2561
1    2165
dtype: int64

In [42]:
from sklearn.ensemble import AdaBoostClassifier 
ada = AdaBoostClassifier(random_state = 42).fit(X_train, y_train)
ada.score(X_test,y_test)*100

55.2735118669882

In [43]:
from xgboost import XGBClassifier
cgb = XGBClassifier(random_state = 42).fit(X_train, y_train)
cgb.score(X_test,y_test)*100

56.68232009138215

# Random Forest(final)

In [44]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier(random_state = 42).fit(X_train, y_train)
forest.score(X_test,y_test)*100

73.930701865719

In [45]:
test_df=pd.read_excel('test.xlsx')
test_df.head()

,ID,Insects,Crop,Soil,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Season
0,1,188,Feed,silt,1,0,NaN,0,2
1,2,410,Feed,silt,1,0,0.0,0,2
2,3,626,Feed,clay,1,0,0.0,0,2
3,4,731,Feed,clay,1,0,0.0,0,2
4,5,789,Food,clay,1,0,0.0,0,1


In [46]:
test_df['Number_of_Weeks_Used'].fillna(test_df.Number_of_Weeks_Used.median(), inplace=True)
test_df2 = pd.get_dummies(test_df, columns=['Crop', 'Soil'])


In [47]:
test_X = test_df2.drop(['ID'],axis = 1)

In [48]:
from sklearn.ensemble import RandomForestClassifier 
forest_final = RandomForestClassifier(random_state = 42).fit(X, y)

In [49]:
forest_final_pred = forest_final.predict(test_X)

In [50]:
pred = pd.Series(forest_final_pred)

In [51]:
pred.value_counts()

0    21652
1    11508
2     1840
dtype: int64

In [52]:
pred.to_csv('Attempt2(Random_forest).csv')

In [59]:
from sklearn.ensemble import VotingClassifier
vot = VotingClassifier(estimators=[('dt',dt),('Foresst',forest)],voting='hard').fit(X, y)
vot_pred = vot.predict(test_X)
pred_vot = pd.Series(vot_pred)
pred_vot.value_counts()

0    24692
1     9277
2     1031
dtype: int64

In [57]:
from sklearn.ensemble import VotingClassifier
vot = VotingClassifier(estimators=[('dt',dt),('Foresst',forest)],voting='soft').fit(X, y)
vot_pred = vot.predict(test_X)
pred_vot = pd.Series(vot_pred)
pred_vot.value_counts()

0    18950
1    13467
2     2583
dtype: int64

In [60]:
pred_vot.to_csv('Attempt4(voth).csv')